In [71]:
import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

In [72]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    base_File = f"{baseFile}/u{fold_Number}.base"
    test_File = f"{baseFile}/u{fold_Number}.test"
    base_Data = pd.read_csv(base_File, sep="\t", header=None, names=rnmColData)
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    base_Data = base_Data.drop(columns=["timestamp"])
    test_Data = test_Data.drop(columns=["timestamp"])
    return base_Data, test_Data

In [73]:
call_base = "ml-100k"
basedata, testdata = load_Dataset(call_base, 1)
testdata


,user_id,item_id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3
...,...,...,...
19995,458,648,4
19996,458,1101,4
19997,459,934,3
19998,460,10,3


In [74]:
# 

In [75]:
# [1:463, 1:1592]

In [76]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((459, 1410)), columns = list(range(1, 1411)), index = list(range(1, 460)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    convertMatriksRating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

In [77]:
calTest = ConvertRatingMatriks(testdata)
calTest

,1,2,3,4,5,6,7,8,9,10,...,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410
1,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def ConvertGTtoList(gt_df):
    groundTruth = []
    # Iterasi untuk setiap user_id di dalam gt_df
    for user_id in gt_df.index:
        # Ambil semua rating user tersebut (baris)
        user_ratings = gt_df.loc[user_id]
        # Ambil item_id yang punya rating > 0 (relevan), ambil nama item koncert ke list
        relevant_items = user_ratings[user_ratings > 0].index.astype(float).tolist()
        # tambahkan ke dalam list groundTruth
        groundTruth.append(relevant_items)
    return groundTruth

In [79]:
rating_Matriks = ConvertRatingMatriks(testdata)
print(rating_Matriks.shape)
# rating_Matriks

# dataset 
allItem = rating_Matriks.columns
alluser = rating_Matriks.index
print(f"all item {len(allItem)}")
itemRated = set(rating_Matriks.columns[rating_Matriks.sum(axis=0) > 0])
print(f"item rated {len(itemRated)}")
userRated = set(rating_Matriks.index[rating_Matriks.sum(axis=1) > 0])
print(f"user rated2 {len(userRated)}")

# item yang tidak pernah di rating
itemNotRated = set(allItem) - itemRated
print(f"item not rated {len(itemNotRated)}")
itemNotRated2 = set(alluser) - userRated
print(f"user not rated2 {len(itemNotRated2)}")

(459, 1410)
all item 1410
item rated 1323
user rated2 457
item not rated 87
user not rated2 2


In [80]:
# toy data ndcg
groundTruth = [[1, 2, 3, 4, 11], [1, 12, 4, 7], [4, 11, 12, 32], [2, 3, 5, 10]]
TopNrek = [[1, 5, 2, 3, 15], [1,4,6,2], [11, 4, 32, 1], [2, 4, 10, 45]]


In [81]:
groundTruth1 = [1, 2, 3, 4]
TopNrek1 = [1, 5, 2, 3, 15]

In [82]:
print(TopNrek[:3])
print(TopNrek[:3][-1] in groundTruth)

[[1, 5, 2, 3, 15], [1, 4, 6, 2], [11, 4, 32, 1]]
False


## DCG

In [83]:
def EvaluasiDCG(groundTruth, TopNRekomendasi, N):
    """
    Fungsi untuk menghitung DCG (Discounted Cumulative Gain) untuk rekomendasi item
    """
    rumusIDCG = np.array([(1 / np.log2( 1 + x)) for x in range(1, N+1)])
    cekGT = np.array([1 if tp in groundTruth else 0 for tp in TopNRekomendasi[:N]])
    return np.sum(rumusIDCG* cekGT)

In [137]:
def dcgN(groundTruth, TopNrek, N):
    totalDCG = 0
    for i in range(len(groundTruth)):
        dcgN = EvaluasiDCG(groundTruth[i], TopNrek[i], N)
        totalDCG += dcgN
        print(f"DCG untuk data ke-{i+1}: {dcgN}")
    avgDCG = totalDCG / len(groundTruth)
    # avgDCG = totalDCG
    print(f"\nRata-rata DCG: {avgDCG}")
    return avgDCG

In [138]:

calDCGN = dcgN(groundTruth, TopNrek, 4)

DCG untuk data ke-1: 1.9306765580733931
DCG untuk data ke-2: 1.6309297535714575
DCG untuk data ke-3: 2.1309297535714578
DCG untuk data ke-4: 1.5

Rata-rata DCG: 1.798134016304077


In [86]:
calDCG = EvaluasiDCG(groundTruth1, TopNrek1, 5)
calDCG

1.9306765580733931

## IDCG

In [87]:
def IdealDCG(N):
    """
    Fungsi untuk menghitung IDCG (Ideal Discounted Cumulative Gain) untuk rekomendasi item
    """
    rumusIDCG = np.sum(np.array([(1 / np.log2( 1 + x)) for x in range(1, N+1)]))
    # print(rumusIDCG)
    return rumusIDCG

In [128]:
calIdcf = IdealDCG(20)
calIdcf

7.040268381923513

## NDCG

In [89]:
def EvaluasiNDCG(groundTruth, TopNRekomendasi, N):
    """
    Fungsi untuk menghitung NDCG (Normalized Discounted Cumulative Gain) untuk rekomendasi item
    """
    evaluasiNDCG = EvaluasiDCG(groundTruth, TopNRekomendasi, N) / IdealDCG(N)
    return evaluasiNDCG

In [90]:
def ndcgN(groundTruth, TopNrek, N):
    totalNDCG = 0
    for i in range(len(groundTruth)):
        ndcgN = EvaluasiNDCG(groundTruth[i], TopNrek[i], N)
        totalNDCG += ndcgN
        print(f"nDCG untuk data ke-{i+1}: {ndcgN}")
    avgNDCG = totalNDCG / len(groundTruth)
    print(f"\nRata-rata NDCG: {avgNDCG}")
    return avgNDCG

In [91]:
calNDCGN = ndcgN(groundTruth, TopNrek, 2)

nDCG untuk data ke-1: 0.6131471927654584
nDCG untuk data ke-2: 1.0
nDCG untuk data ke-3: 1.0
nDCG untuk data ke-4: 0.6131471927654584

Rata-rata NDCG: 0.8065735963827292


In [92]:
calNDCG = EvaluasiNDCG(groundTruth1, TopNrek1, 4)
calNDCG

0.75369761125927

## LOPP TEST REAL DATA

In [111]:
# openTopN = joblib.load("case/topN/testTopn1.joblib")
openTopN2 = joblib.load("case/topN/testTopn2.joblib")

# jadikan data frame
print(len(openTopN2))

459


In [94]:
convertGt = ConvertGTtoList(calTest)
print(len(convertGt))

459


In [131]:
calDCGN = dcgN(convertGt, openTopN2, 20)


DCG untuk data ke-1: 0.27023815442731974
DCG untuk data ke-2: 0.0
DCG untuk data ke-3: 0.0
DCG untuk data ke-4: 0.0
DCG untuk data ke-5: 0.0
DCG untuk data ke-6: 0.5066131943480828
DCG untuk data ke-7: 1.3584156399459009
DCG untuk data ke-8: 0.227670248696953
DCG untuk data ke-9: 0.0
DCG untuk data ke-10: 0.0
DCG untuk data ke-11: 0.26264953503719357
DCG untuk data ke-12: 0.0
DCG untuk data ke-13: 2.0696292023716283
DCG untuk data ke-14: 0.0
DCG untuk data ke-15: 0.0
DCG untuk data ke-16: 0.746277808543962
DCG untuk data ke-17: 0.0
DCG untuk data ke-18: 0.0
DCG untuk data ke-19: 0.0
DCG untuk data ke-20: 0.0
DCG untuk data ke-21: 0.27894294565112987
DCG untuk data ke-22: 0.8586000022684105
DCG untuk data ke-23: 0.0
DCG untuk data ke-24: 0.0
DCG untuk data ke-25: 0.0
DCG untuk data ke-26: 0.0
DCG untuk data ke-27: 0.0
DCG untuk data ke-28: 0.0
DCG untuk data ke-29: 0.0
DCG untuk data ke-30: 0.0
DCG untuk data ke-31: 0.0
DCG untuk data ke-32: 0.0
DCG untuk data ke-33: 0.0
DCG untuk data 

In [120]:
calNDCGN = ndcgN(convertGt, openTopN2, 20)
    

nDCG untuk data ke-1: 0.038384638165382894
nDCG untuk data ke-2: 0.0
nDCG untuk data ke-3: 0.0
nDCG untuk data ke-4: 0.0
nDCG untuk data ke-5: 0.0
nDCG untuk data ke-6: 0.0719593582041354
nDCG untuk data ke-7: 0.1929494113368957
nDCG untuk data ke-8: 0.03233829114831981
nDCG untuk data ke-9: 0.0
nDCG untuk data ke-10: 0.0
nDCG untuk data ke-11: 0.03730675036644463
nDCG untuk data ke-12: 0.0
nDCG untuk data ke-13: 0.2939702139318406
nDCG untuk data ke-14: 0.0
nDCG untuk data ke-15: 0.0
nDCG untuk data ke-16: 0.10600132950330327
nDCG untuk data ke-17: 0.0
nDCG untuk data ke-18: 0.0
nDCG untuk data ke-19: 0.0
nDCG untuk data ke-20: 0.0
nDCG untuk data ke-21: 0.0396210670558156
nDCG untuk data ke-22: 0.12195557835166326
nDCG untuk data ke-23: 0.0
nDCG untuk data ke-24: 0.0
nDCG untuk data ke-25: 0.0
nDCG untuk data ke-26: 0.0
nDCG untuk data ke-27: 0.0
nDCG untuk data ke-28: 0.0
nDCG untuk data ke-29: 0.0
nDCG untuk data ke-30: 0.0
nDCG untuk data ke-31: 0.0
nDCG untuk data ke-32: 0.0
nDCG

In [132]:
# ambil rata2 dcg 

hitungNDCG = (calDCGN /calIdcf)
print(f"fold=1 {hitungNDCG}")

fold=1 0.02955399405008243
